# Load images stored

In [ ]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50
import tensorflow as tf
from keras import losses
import keras.backend as K
import tensorflow as tf
from keras.models import model_from_json

In [ ]:
def load_images(train, directory):
    
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 224
    train_image = []
    for name in train['image_id']:
        path = directory + 'images/' + name + '.jpg'
        img = cv2.imread(path)
        img = cv2.resize(img, (img_size, img_size))
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [ ]:
def load_images_augmented(directory):
    
    # This function loads the augmented images and the augmented csv file
    df_train = pd.read_csv(directory + 'augmented.csv')
    train_image = []
    for name in df_train['image_id']:
        name = name.lower()
        path = directory + 'images_resized_augmented/' + name + '.jpg'
        img = cv2.imread(path)
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return  train_image_array, df_train

In [ ]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
x_train, df_train = load_images_augmented(directory)

In [ ]:
# Normalize
x_train = x_train / 255.0 

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y_train = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].to_numpy()

x_train_original, y_train_original = shuffle(x_train, y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train_original, y_train_original, test_size = 0.2, random_state = 2020)

print('Size of x_train: ', x_train.shape)
print('Size of x_val: ', x_val.shape)

In [ ]:
def predict_and_save(model, x_test, name):
    x_pred = model.predict(x_test)
    df_test['healthy'] = x_pred[:,0]
    df_test['multiple_diseases'] = x_pred[:,1]
    df_test['rust'] = x_pred[:,2]
    df_test['scab'] = x_pred[:,3]
    df_test.to_csv(name, index = None)

In [ ]:
def model_plot(history):

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
def print_score(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

# Localization, Pooling and Embedding

In [ ]:
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.legacy import interfaces




class GlobalKMaxPooling2D(Layer): #Inherits the properties of Layer class
    """K Max Pooling operation for spatial data.
    
    # Arguments
        
        data_format: A string,
            one of `"channels_last"` (default) or `"channels_first"`.
            The ordering of the dimensions in the inputs.
            `"channels_last"` corresponds to inputs with shape
            `(batch, height, width, channels)` while `"channels_first"`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be `"channels_last"`.
            
        K: An Integer,
            states the number of selected maximal values over which the
            average is going to be computed.
            
            
    # Input shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, rows, cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, rows, cols)`
            
    # Output shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, pooled_rows, pooled_cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, pooled_rows, pooled_cols)`    
    
    
    """
    

    def __init__(self, data_format=None, k = 10, **kwargs):
        super(GlobalKMaxPooling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        self.k = k

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def get_config(self):
        config = {'data_format': self.data_format, 'k' : self.k}
        base_config = super(GlobalKMaxPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
        
    def call(self, inputs):
        if self.data_format == 'channels_last':
            # Here first sort
            # Then take K maximum values
            # Then average them
            k = self.k

            input_reshaped = tf.reshape(inputs, [tf.shape(inputs)[0], -1, tf.shape(inputs)[3]])
            input_reshaped = tf.reshape(input_reshaped, [tf.shape(input_reshaped)[0], tf.shape(input_reshaped)[2], tf.shape(input_reshaped)[1]])
            top_k = tf.math.top_k(input_reshaped, k=k, sorted = True, name = None)[0]
            mean = tf.keras.backend.mean(top_k, axis = 2)
            #assert ((input_reshaped.get_shape()[0], input_reshaped.get_shape()[-1]) == mean.get_shape())
        
        return mean

    
    
    

In [ ]:
def build_lrfn(lr_start=0.00001, lr_max=0.00005, 
               lr_min=0.00001, lr_rampup_epochs=5, 
               lr_sustain_epochs=0, lr_exp_decay=.8):
    lr_max = lr_max #* strategy.num_replicas_in_sync

    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) *\
                 lr_exp_decay**(epoch - lr_rampup_epochs\
                                - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

In [ ]:
lrfn = build_lrfn()
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [ ]:
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50

In [ ]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [ ]:

model = Sequential()
model.add(model_resnet)
model.add(GlobalKMaxPooling2D(data_format = 'channels_last' , k = 4))
# model.add(Dense(128, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = EmbeddingLayerLoss(model.layers[2]), metrics = ['categorical_accuracy'])
print(model.summary())


In [ ]:
model.layers[-2]


In [ ]:
y_train.shape

In [ ]:
from keras.applications.resnet import ResNet50
from keras.losses import categorical_crossentropy
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time


    
class ElopeModel(object):
    
    def __init__(self):
        
        # Model
        self.model = self.build_model()
        # Print a model summary
        self.model.summary()
        
        # Optimizer
        self.optimizer = 'Adam'
        
        # Have to convert to tensor??
        self.loss_parameters = {'means' : 0, 'lr' : tf.constant(0.5,tf.float32), 'landa' : tf.constant(2.0,tf.float32),
                               'gamma' : tf.constant(16.0,tf.float32), 'm' : tf.constant(0.75,tf.float32)}

        
        #Loss Function
        self.loss_func = self.model_loss()
        
        self.compile()
        
        
    def build_model(self):
        
        input_shape = (224, 224, 3)
        model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)
        model = Sequential()
        
        model.add(model_resnet)
        model.add(GlobalKMaxPooling2D(data_format = 'channels_last' , k = 4))
        model.add(Dense(4, activation = 'softmax'))
        tf.compat.v1.enable_eager_execution()
        print('Eager execution:', tf.executing_eagerly())
        
        return model
    
    
    def calculate_within_class_loss(self, y_true, y_pred, fx_tensor): #fx_tensor = Output of layer
       
        mean_tensor = self.loss_parameters['means']
        alpha = self.loss_parameters['lr']

        dimension_tensor = K.shape(fx_tensor)[1]
        num_classes_tensor = K.shape(y_true)[1]
        if isinstance(mean_tensor, int): # Initialize them if they are not yet initialized
            mean_tensor = tf.zeros([num_classes_tensor, dimension_tensor], dtype = tf.dtypes.float32)
        num_samples_tensor = K.shape(fx_tensor)[0]
        
        # Ensure they are float 32
        fx_tensor = tf.cast(fx_tensor, dtype = tf.dtypes.float32)
        

        fx_expanded = tf.broadcast_to(tf.expand_dims(fx_tensor, axis = -1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
        y_true_expanded = tf.broadcast_to(tf.expand_dims(y_true, axis = 1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
        mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])

        mean_expanded = tf.transpose(mean_expanded, perm = [0,2,1])

        up = tf.reduce_sum(tf.multiply(tf.subtract(mean_expanded, fx_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float32)), axis = 0)
        y_true_cut = tf.cast(tf.reduce_sum(y_true, axis = 0), dtype = tf.dtypes.float32)
        down = tf.add(y_true_cut, tf.constant(1, dtype = tf.dtypes.float32))
        delta = tf.divide(up, down)
        delta = tf.transpose(delta)

        mean_new = tf.subtract(mean_tensor, tf.scalar_mul(alpha, delta))

        # Now calculate the loss
        mean_new_expanded = tf.broadcast_to(tf.expand_dims(mean_new, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])
        mean_new_expanded = tf.transpose(mean_new_expanded,  perm = [0,2,1]) 
        inside = tf.reduce_sum(tf.multiply(tf.subtract(fx_expanded, mean_new_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float32)), axis = 2)


        tot = tf.reduce_sum(tf.multiply(inside, inside)) # Apply the norm
        down = tf.multiply(tf.constant(2), num_samples_tensor)
        loss = tf.divide(tot, tf.cast(down, dtype = tf.dtypes.float32))

        self.loss_parameters['means'] = mean_new
        
        
        # N = number of samples in the batch
        # f(xn) -> Dimension = 2048 (not the batch because it it xn, only 1 sample)
        # class means -> should be the same as f(xn) x number of classes
        # Therefore ||f(xn) - U(cn)||^2 will be a number, because f(xn) is a vector, not a matrix

        
        return loss
    
    
    
    def calculate_between_class_loss(self, y_true, y_pred, fx_tensor):
        # Layer there or in self??
        mean_tensor = self.loss_parameters['means']
        gamma = self.loss_parameters['gamma']
        m = self.loss_parameters['m']
        # m -> Margin
        # P -> class-pairs in the current batch
        # |P| -> Cardinality of P, number of elements in the set
        
        # print(mean_tensor.numpy())

        dimension_tensor = K.shape(fx_tensor)[1]
        num_classes_tensor = K.shape(y_true)[1]
        num_samples_tensor = K.shape(y_true)[0]

        # First we have to expand the mean matrix in order to substract it
        mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_classes_tensor, num_classes_tensor, dimension_tensor])
        # We transpose it to do the subtraction
        mean_trans = tf.transpose(mean_expanded, [1,0,2])

        # We subtract it and do the norm through the "dimensions" axis.
        norm = tf.subtract(mean_expanded, mean_trans)
        norm = tf.reduce_sum(tf.multiply(norm, norm), axis = 2) 

        # Finally we subtract to m the previously calculated norm vector
        m_tensor = tf.scalar_mul(m, tf.ones(K.shape(norm), tf.float32))
        inside = tf.subtract(m_tensor, tf.cast(norm, tf.float32))

        # Now we only take the lower part diagonal matrix, and as we cannot delete the diagonal, we subtract it in the final

        mat = tf.linalg.LinearOperatorLowerTriangular(inside)
        diagonal = mat.diag_part() # diagonal
        mat = mat.to_dense() # Lower triangular matrix with diagonal

        zeros = tf.zeros(K.shape(norm), tf.float32)
        maximum = tf.maximum(mat, zeros)
        max_squared = tf.square(maximum)

        summ = tf.reduce_sum(max_squared)

        diag_sum = tf.reduce_sum(tf.multiply(diagonal, diagonal))

        summ = tf.subtract(summ, diag_sum)



        counter = tf.ones([num_classes_tensor, num_classes_tensor], tf.float32)
        counter = tf.linalg.LinearOperatorLowerTriangular(counter)

        count_diag = counter.diag_part()
        count_triang = counter.to_dense()
        counter = tf.subtract(tf.reduce_sum(count_triang), tf.reduce_sum(count_diag))


        loss = tf.multiply(tf.divide(gamma, tf.multiply(tf.constant(4, tf.float32), counter)), summ)
        
        return loss
    
    
    
    
    def model_loss(self):
        """" Wrapper function which calculates auxiliary values for the complete loss function.
         Returns a *function* which calculates the complete loss given only the input and target output """
        # This part has to be developed
        
        #Within class loss
        within_class_loss_func = self.calculate_within_class_loss
        
        # Between class loss
        between_class_loss_func = self.calculate_between_class_loss
        
        lay_out = self.model.layers[-2].output
        
        
        def ElopeLoss(y_true, y_pred):
            
            # Within Class loss has to be computed first, in order to get the new class means updated
            
            within_class_loss = within_class_loss_func(y_true, y_pred, lay_out)
            
            between_class_loss = between_class_loss_func(y_true, y_pred, lay_out)
            
            cat_cross_loss = categorical_crossentropy(y_true, y_pred)
            
            model_loss = cat_cross_loss + within_class_loss + between_class_loss
            
            return model_loss
        
        
        return ElopeLoss
    
    
    def compile(self):
        """ Compiles the Keras model. Includes metrics to differentiate between the two main loss terms """
        self.model.compile(optimizer = self.optimizer, loss = self.loss_func,
                          metrics = [categorical_crossentropy]) # Here put the two losses better
        print('Model Compiled!')
        
    
    def load_trained_weights(self, weights):
        """ Loads weights of a pre-trained model. 'weights' is path to h5 model\weights file"""
        self.model.load_weights(weights)
        print('Weights from {} loaded successfully'.format(weights))
        

In [ ]:
Elope = ElopeModel()

In [ ]:
history = Elope.model.fit(x_train[:120], y_train[:120], batch_size = 12, epochs = 1,
                    validation_data = (x_val[:2], y_val[:2]), callbacks = [lr_schedule])

In [ ]:
json_file = open('testing.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json, custom_objects = {'GlobalKMaxPooling2D': GlobalKMaxPooling2D})
# load weights into new model
loaded_model.load_weights("testing.h5")
print("Loaded model from disk")

In [ ]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
df_test = pd.read_csv(directory + 'test.csv')
x_test = load_images(df_test, directory)

In [ ]:
predict_and_save(loaded_model, x_test, name = 'submit_test.csv')

# Verification of Between Class loss and Within Class loss

In [ ]:
# First we are going to specify some random values and convert them to tensors

f_x = np.random.rand(3,2048)
mean = np.random.rand(4,2048)
y_true = np.array([[0,0,1,0], [1,0,0,0], [0,1,0,0]])

fx_tensor = tf.convert_to_tensor(f_x)
mean_tensor = tf.convert_to_tensor(mean)
y_true_tensor = tf.convert_to_tensor(y_true)

In [ ]:
fx_tensor.shape[1]

In [ ]:
# Now we are going to get the necessary slices from the tensors

fx_tensor_slice = tf.slice(f_x, [1,0], [1, tf.shape(fx_tensor)[1].numpy()]) #Begin, size
y_true_slice = tf.slice(y_true, [1,0], [1,4])
class_number = np.argmax(y_true_slice.numpy()) # Class goes from 0 to 3, in order to fit better into slice method
mean_slice = tf.slice(mean_tensor, [class_number, 0], [1,tf.shape(mean_tensor)[1].numpy()])

In [ ]:
print('f_x shape: ', tf.shape(fx_tensor_slice))
print('mean shape: ', tf.shape(mean_slice) )
print('y_true shape: ', tf.shape(y_true_slice))

In [ ]:
mean_slice - fx_tensor_slice

In [ ]:
K.eval(fx_tensor)

# Calculate loss with graph mode

## Within class loss

In [ ]:
alpha = 0.5
num_classes = mean_tensor.shape[0]
num_samples_in_batch = fx_tensor.shape[0]
dimensions = mean_tensor.shape[1]

# print('mean_tensor shape: ', mean_tensor.shape)
# print('fx_tensor shape: ', fx_tensor.shape)
# print('y_true shape: ', y_true.shape)

dimension_tensor = K.shape(fx_tensor)[1]
num_classes_tensor = K.shape(y_true)[1]
num_samples_tensor = K.shape(y_true)[0]

fx_expanded = tf.broadcast_to(tf.expand_dims(fx_tensor, axis = -1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
y_true_expanded = tf.broadcast_to(tf.expand_dims(y_true_tensor, axis = 1), [num_samples_tensor, dimension_tensor, num_classes_tensor])
mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])

mean_expanded = tf.transpose(mean_expanded, perm = [0,2,1])

up = tf.reduce_sum(tf.multiply(tf.subtract(mean_expanded, fx_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 0)

y_true_cut = tf.reduce_sum(y_true_tensor, axis = 0)
down = tf.add(y_true_cut, tf.constant(1))
down = tf.cast(down, dtype = tf.dtypes.float64)
delta = tf.divide(up, down)


delta = tf.transpose(delta)

mean_new = tf.subtract(mean_tensor, tf.scalar_mul(tf.constant(alpha, dtype = tf.dtypes.float64), delta))

mean_new_sum = np.sum(mean_new.numpy())
print()
print(mean_new_sum)
print(mean_new.numpy())
print()
print('---------------- NOW THE LOSS ---------------')
print()

# Now calculate the loss
mean_new_expanded = tf.broadcast_to(tf.expand_dims(mean_new, axis = 0), [num_samples_tensor, num_classes_tensor, dimension_tensor])
mean_new_expanded = tf.transpose(mean_new_expanded,  perm = [0,2,1]) 

print()
inside = tf.reduce_sum(tf.multiply(tf.subtract(fx_expanded, mean_new_expanded), tf.cast(y_true_expanded, dtype = tf.dtypes.float64)), axis = 2)


tot = tf.reduce_sum(tf.multiply(inside, inside)) # Apply the norm
down = tf.multiply(tf.constant(2), num_samples_tensor)
tot = tf.divide(tot, tf.cast(down, dtype = tf.dtypes.float64))

print(tot.numpy())

# mean_tensor = mean_new

## Between class loss

In [ ]:
m = 1000
gamma = tf.constant(16, tf.float64)
# print(mean_tensor.numpy())

dimension_tensor = K.shape(fx_tensor)[1]
num_classes_tensor = K.shape(y_true)[1]
num_samples_tensor = K.shape(y_true)[0]
print()
print('dimension:', dimension_tensor.numpy())
print('num_classes:', num_classes_tensor.numpy())
print('num_samples:', num_samples_tensor.numpy())
print()

# First we have to expand the mean matrix in order to substract it
mean_expanded = tf.broadcast_to(tf.expand_dims(mean_tensor, axis = 0), [num_classes_tensor, num_classes_tensor, dimension_tensor])
# We transpose it to do the subtraction
mean_trans = tf.transpose(mean_expanded, [1,0,2])

# We subtract it and do the norm through the "dimensions" axis.
norm = tf.subtract(mean_expanded, mean_trans)
norm = tf.reduce_sum(tf.multiply(norm, norm), axis = 2) 

# Finally we subtract to m the previously calculated norm vector
m_tensor = tf.scalar_mul(tf.constant(m, tf.float64), tf.ones(norm.shape, tf.float64))
inside = tf.subtract(m_tensor, norm)


# print(inside.numpy())
print()

# Now we only take the lower part diagonal matrix, and as we cannot delete the diagonal, we subtract it in the final
print()
mat = tf.linalg.LinearOperatorLowerTriangular(inside)
diagonal = mat.diag_part() # diagonal
mat = mat.to_dense() # Lower triangular matrix with diagonal

zeros = tf.zeros(norm.shape, tf.float64)
maximum = tf.maximum(mat, zeros)
max_squared = tf.square(maximum)
# print(max_squared.numpy())
summ = tf.reduce_sum(max_squared)
# print('After ')
# print(summ.numpy())
diag_sum = tf.reduce_sum(tf.multiply(diagonal, diagonal))
# print(diag_sum.numpy())
summ = tf.subtract(summ, diag_sum)
# print(summ.numpy())


counter = tf.ones([num_classes_tensor, num_classes_tensor], tf.float64)
counter = tf.linalg.LinearOperatorLowerTriangular(counter)

count_diag = counter.diag_part()
count_triang = counter.to_dense()
counter = tf.subtract(tf.reduce_sum(count_triang), tf.reduce_sum(count_diag))


loss = tf.multiply(tf.divide(gamma, tf.multiply(4, counter)), summ)


# loss = (gamma/(4*counter)) *  summ.numpy()


print(loss.numpy())







## Implement it in Numpy to see it is correct

In [ ]:
# First we are going to specify some random values and convert them to tensors

f_x = np.random.rand(6,2048) 
mean = np.random.rand(4,2048) 
y_true = np.array([[0,0,1,0], [1,0,0,0], [0,1,0,0], [0,0,0,1], [1,0,0,0], [0,1,0,0]])

# f_x = np.array([[2,3,4,5], [-1, 0, -2, -3], [5,6, 8, 6]], dtype = 'float64')
# mean = np.array([[-2, -1, -3, -2], [4, 5, 7, 6]], dtype = 'float64')
# y_true = np.array([[0,1], [1,0], [0,1]])

fx_tensor = tf.convert_to_tensor(f_x, dtype = tf.dtypes.float64)
mean_tensor = tf.convert_to_tensor(mean, dtype = tf.dtypes.float64)
y_true_tensor = tf.convert_to_tensor(y_true)


In [ ]:
# This is well done!
alpha = 0.5
num_classes = 2
num_samples_in_batch = 3
dimension = 4

# LOOP THROUGH CLASSES TO CALCULATE NEW MEAN
for i in range(0, num_classes): # Loops throgh the classes

    counter = 1 # to divide it
    summ = np.zeros(dimension)  # Initialize class means 
    mean_i = mean[i]
    for j in range(0, num_samples_in_batch): # Loops through the images in the batch  
        y_true_sample = y_true[j] # Class of the i image
        class_number = np.argmax(y_true_sample) 
        
        if class_number == i: # if not, do nothing
            counter += 1
            summ = summ + (mean_i - f_x[j])
        
    total = summ / counter
    mean[i] = mean_i - alpha*total 
    
print('The new mean: ', mean)
print()

# Calculate the loss
suma = 0
for i in range(0, num_samples_in_batch):
    y_true_sample = y_true[i] # Class of the i image
    class_number = np.argmax(y_true_sample)
    inside = f_x[i] - mean[class_number]
    norm = np.linalg.norm(inside)
    squared = np.square(norm)
    suma = suma + np.square(np.linalg.norm(f_x[i] - mean[class_number]))
loss = suma / (2*num_samples_in_batch)
print('The final loss:', loss)


# Implement the Between Class loss

In [ ]:
m = 1000
summ = 0 
counter = 0
gamma = 16
# print(mean)
print()
for i in range(0, len(mean)):
    for j in range(i, len(mean)):
        if i != j:
            summ = summ + np.square(max(m - np.square(np.linalg.norm(mean[i] - mean[j])), 0))
#             print(mean[i])
#             print(mean[j])
#             print()
            counter += 1
loss = (gamma/(4*counter)) * summ
print(counter)
loss